---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd
import re

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
def date_sorter():
    
    # Your code here
    
    return # Your answer here

In [3]:
#df2 = df.str.findall(r'(?:\d{1,2}\W*)?(?:\d+[a-z]{2}\W*)?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\W*(?:\d{1,2}\W*)?(?:[a-z]{2}\W*)?\d{4}|(?:\d{1,2}[/-]\d{1,2}?[/-]?\d{2,4})|(?:19|20)[0-9]{2}')
# consider 271 and same positions
df2 = df.str.findall(r'(?:\d{1,2}\W*)?(?:\d+[a-z]{2}\W*)?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\W*(?:\d{1,2}\W*)?(?:[a-z]{2}\W*)?\d{4}|(?:\d{1,2}[/-]\d{1,2}[/-]\d{2,4})|(?:\d{1,2}[/-][1-2]\d\d{2})|(?:19|20)[0-9]{2}')

In [4]:
# All values are a lists (with only one element, extract them from the list)
df2 = df2.apply(lambda x: x[0])

In [5]:
df2.shape

(500,)

In [6]:
# Series to dataframe
df2 = df2.to_frame()

### Extract the Year

In [7]:
#df2.index.values
# extract the year
df2['year'] = df2[0].str.extractall(r'(?P<year>(?P<year1>\d{4})|(?P<year2>\d{2})$)')['year'].values

In [8]:
df2.head()

,0,year
0,03/25/93,93
1,6/18/85,85
2,7/8/71,71
3,9/27/75,75
4,2/6/96,96


In [9]:
df2['year'][df2.year.notnull()].shape

(500,)

In [10]:
df2['year'] = df2['year'].apply(lambda x: int(x) + 1900 if int(x)<100 else int(x))

In [11]:
# check the shape that all columns are there
df2.shape

(500, 2)

### Extract the Month

In [12]:
month = df2[0].str.extractall(r'((?P<month1>\d{1,2})\W\d{1,2}\W|(?P<month2>^\d{1,2})\W\d{4}|(?P<month3>[a-zA-Z]{3,})|(?P<month4>^\W*\d{4}\W*$))')

In [13]:
month.head()

,,0,month1,month2,month3,month4
,match,,,,,
0,0,03/25/,03,NaN,NaN,NaN
1,0,6/18/,6,NaN,NaN,NaN
2,0,7/8/,7,NaN,NaN,NaN
3,0,9/27/,9,NaN,NaN,NaN
4,0,2/6/,2,NaN,NaN,NaN


In [14]:
# Create a dictionary for month
m = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}

In [15]:
month['month1'] = month.month1.apply(lambda x: int(x) if type(x)==int or type(x)==str else 0)
month['month2'] = month.month2.apply(lambda x: int(x) if type(x)==int or type(x)==str else 0)
month['month3'] = month.month3.apply(lambda x: m[x[:3]] if type(x)==str else 0)
month['month4'] = month.month4.apply(lambda x: 1 if type(x)==int or type(x)==str else 0)

In [16]:
month.head()

,,0,month1,month2,month3,month4
,match,,,,,
0,0,03/25/,3,0,0,0
1,0,6/18/,6,0,0,0
2,0,7/8/,7,0,0,0
3,0,9/27/,9,0,0,0
4,0,2/6/,2,0,0,0


In [17]:
df2['month'] = 0

In [18]:
df2.iloc[month.index.levels[0][month.month1!=0].tolist(),2] = month['month1'][month.month1!=0].values
df2.iloc[month.index.levels[0][month.month2!=0].tolist(),2] = month['month2'][month.month2!=0].values
df2.iloc[month.index.levels[0][month.month3!=0].tolist(),2] = month['month3'][month.month3!=0].values
df2.iloc[month.index.levels[0][month.month4!=0].tolist(),2] = month['month4'][month.month4!=0].values

In [19]:
set(df2['month'].values)

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}

In [20]:
df2['month'][df2.month==0]

Series([], Name: month, dtype: int64)

### Extract the Day

In [21]:
day = df2[0].str.extractall(r'[/-](?P<day1>\d{1,2})[-/]\d+|[a-zA-Z]+\W*(?P<day2>\d{1,2})\W+|(?P<day3>\d{1,2})\W*[a-zA-Z]+')

In [22]:
day.head()

,,day1,day2,day3
,match,,,
0,0,25,NaN,NaN
1,0,18,NaN,NaN
2,0,8,NaN,NaN
3,0,27,NaN,NaN
4,0,6,NaN,NaN


In [23]:
df2['day'] = 1

In [24]:
day['day1'] = day.day1.apply(lambda x: int(x) if type(x)==int or type(x)==str else 0)
day['day2'] = day.day2.apply(lambda x: int(x) if type(x)==int or type(x)==str else 0)
day['day3'] = day.day3.apply(lambda x: x if type(x)==str else 0)

In [25]:
df2.iloc[day.index.levels[0][day.day1!=0].tolist(),3] = day['day1'][day.day1!=0].values
df2.iloc[day.index.levels[0][day.day2!=0].tolist(),3] = day['day2'][day.day2!=0].values
df2.iloc[day.index.levels[0][day.day3!=0].tolist(),3] = day['day3'][day.day3!=0].values

In [26]:
df2.head()

,0,year,month,day
0,03/25/93,1993,3,25
1,6/18/85,1985,6,18
2,7/8/71,1971,7,8
3,9/27/75,1975,9,27
4,2/6/96,1996,2,6


In [27]:
# return the answer as a Pandas Series
pd.Series(df2.sort_values(['year','month','day']).index.tolist())

0        9
1       84
2        2
3       53
4       28
5      474
6       13
7      153
8       98
9      129
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64

## Appendix A
### Check the regex below using the data from the task 

In [28]:
text1 = '04/20/2009; 04/20/09; 4/20/09; 4/3/09; 6/2008; 12/2009; 11-12-2009; 6/2008; 12/2009; Mar. 20, 2009; B12 969 2007'

In [29]:
re.findall(r'(?:\d{1,2}[/-]\d{1,2}[/-]\d{2,4})|(?:\d{1,2}[/-][1-2]\d\d{2})|(?:19|20)[0-9]{2}', text1)

['04/20/2009',
 '04/20/09',
 '4/20/09',
 '4/3/09',
 '6/2008',
 '12/2009',
 '11-12-2009',
 '6/2008',
 '12/2009',
 '2009',
 '2007']

In [30]:
text2 = 'Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009; 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009'

In [31]:
re.findall(r'(?:\d{1,2}\W*)?(?:\d+[a-z]{2}\W*)?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\W*(?:\d{1,2}\W*)?(?:[a-z]{2}\W*)?\d{4}', text2)

['Mar-20-2009',
 'Mar 20, 2009',
 'March 20, 2009',
 'Mar. 20, 2009',
 'Mar 20 2009',
 '20 Mar 2009',
 '20 March 2009',
 '20 Mar. 2009',
 '20 March, 2009']

In [32]:
text3 = 'Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009, Feb 2009; Sep 2009; Oct 2010'

In [33]:
re.findall(r'(?:\d{1,2}\W*)?(?:\d+[a-z]{2}\W*)?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\W*(?:\d{1,2}\W*)?(?:[a-z]{2}\W*)?\d{4}', text3)

['Mar 20th, 2009',
 'Mar 21st, 2009',
 'Mar 22nd, 2009',
 'Feb 2009',
 'Sep 2009',
 'Oct 2010']